# **Setup the development framework such that changes in py files auto load into this file**
- You can ignore this if your interest is only in the business logic and design. 
- The steps in this section are only environment set related

### **Make sure py file change are loaded automatically**

In [1]:
%load_ext autoreload
%autoreload 2

### **Import basic package required for this assignment**

In [2]:
import os
import pandas as pd
import db.constants as C


### **Make sure right sources are getting picked**

In [3]:
from dao.base_transformer import BaseDBTransformer

### **Make sure the latest class are getting loaded.**

In [4]:
# 1. Import the modules first (so Python knows them)
import importlib
import service.product_transformer
import service.cart_transformer
import service.purchase_transformer
import service.discount_transformer
import db.constants
import db.dbsql

# 2. Reload them (useful after edits)
importlib.reload(service.product_transformer)
importlib.reload(service.cart_transformer)
importlib.reload(service.purchase_transformer)
importlib.reload(service.discount_transformer)
importlib.reload(db.constants)

# 3. Import the classes fresh from the reloaded modules
from service.product_transformer import ProductTransformer
from service.cart_transformer import CartTransformer
from service.purchase_transformer import PurchaseTransformer
from service.discount_transformer import DiscountTransformer
import db.constants
import random

# **Transformers**
- This section is the core design of the project
- There are four transformers 
   - Products transformer: This is the database of the list of the products in EStore. The prodcuts.csv file for demo has 9 products. 
   - Cart transformer: This is user selected products added into the cart instance specific to the user. 
   - purchase transfromer: This is checkout functionaility of the cart. 
        - It has two kinds of purchases. With discount by applying discount code.
        - It also supports purchase without applying discount code. If no discount is applied it generates a ne discount code and adds to discounts for the subsequent purchases. 
   - Discount transformer: This handles the discounts codes after the purchase. 

   - For each these transforers there is .py  file and .csv file in the src directory. 
   - Each transfomer derives from the base transfomer class which manages the csv read/write for that transformer. 
   - The base transformer class also handles basis CRUD operation if they are at the level of the dataframe(entire row level operations) without having to deal with individual columns (as columns are specific to the transformer)

- The code is organized as a typical pyproject.toml file. 

## **Product Transformer - demonstration**

In [ ]:
pt = ProductTransformer()

pt.list_products_in_class(2050)

In [ ]:
# pt = ProductTransformer(engine)

# Load existing rows into memor
df = pt.read()

# Insert new product
r = pt.create(301, "IBM Thinkpad Laptop", 2050, 5000.00, 100, 150, 300, 20, 1.50)
print(r)
# pt.create(302, "IBM Thinkpad Laptop", 2050, 5000.00, 100, 150, 300, 20, 1.50)


# Update a product
# pt.update(301, **{C.mrp:4999.99} )

# Delete a product
# pt.delete(302)


In [ ]:
pt.data

## **Discount Transformer - demonstration**

In [ ]:
# Initialize
discount_csv = "data/discounts.csv"
discount_transformer = DiscountTransformer(file_path=discount_csv)

# Create a random data
cid1 = "CART" + str(random.randint(100, 999))  # Generate a random cart ID
cid2 = "CART" + str(random.randint(100, 999))  # Generate a random cart ID

user_id1 = str(random.randint(100, 999))
user_id2 = str(random.randint(100, 999))

discode1 = [ "NEW", "SAVE", "FESTIVE"][random.randint(0, 2)] + str(random.randint(1000, 9999))
discode2 = [ "NEW", "SAVE", "FESTIVE"][random.randint(0, 2)] + str(random.randint(1000, 9999))

did1 = ["FD","NC","DR"][random.randint(0,2)] + str(random.randint(100, 999))
did2 = ["FD","NC","DR"][random.randint(0,2)] + str(random.randint(100, 999))

dpct1 = random.randint(5, 10)
dpct2 = random.randint(5, 10)


# Test: Write sample data
sample_discounts = pd.DataFrame([
    {C.did: did1, C.usrid: user_id1, C.dcode: discode1, C.dpct: dpct1, C.dst: 0},
    {C.did: did2, C.usrid: user_id2, C.dcode: discode2, C.dpct: dpct2, C.dst: 0}
])

discount_transformer.update_df(sample_discounts, mode="overwrite")
discount_transformer.read()

## **Purchase Transformer - demonstration**

In [ ]:
# Initialize
purchase_csv = "data/purchases.csv"
purchase_transformer = PurchaseTransformer(file_path=purchase_csv)

# Test: Write sample data
sample_purchases = pd.DataFrame([
    {C.prcid: cid1+"_"+user_id1, C.cid: cid1, C.tamt: 772.92, C.did: did1},
    {C.prcid: cid2+"_"+user_id2, C.cid: cid2, C.tamt: 972.00, C.did: did2}
])
purchase_transformer.update_df(sample_purchases, mode="overwrite")
purchase_transformer.read()

## **Cart Transformer - demonstration**

In [ ]:
# Initialize
cart_csv = "data/carts.csv"
cart_transformer = CartTransformer(file_path=cart_csv)

# Test: Write sample data
sample_cart = pd.DataFrame([
    {C.cid: cid1 , C.pid: "FD7839", C.qnt: 4, C.usrid: user_id1},
    {C.cid: cid2 , C.pid: "DR7979", C.qnt: 2, C.usrid: user_id2}
])
cart_transformer.update_df(sample_cart, mode="overwrite")

# Test: Read back
print(cart_transformer.read())


# **Business logic - Helper functions**
- These functions after matured will go into the respective transformer classes. 

In [ ]:
def getRandomPids():
    products = product_transformer.read()

    selected_products = random.sample(products[C.pid].to_list(), k=3)
    print("Selected Products:", selected_products)
    return selected_products

def addToCart(spids, uid, cid):
    cart = cart_transformer.read()
    products = product_transformer.read(spids)

    records = [{C.cid: cid, C.qnt: random.randint(1, 5),C.usrid: uid, C.pid: id} for id in spids]
    cart = pd.DataFrame.from_records(records)
    cart_transformer.create(cart)
    return

def checkout_with_discount(uid, cid, did):
    cart     =  cart_transformer.read([cid])
    ps  =  product_transformer.read(cart[C.pid].to_list())

    # Compute total price for the cart
    total = ps[C.mrp].sum() * cart[C.qnt].sum()
    prcid = uid + "_" + cid  # Purchase ID based on user and cart ID
    print ( "Total price for cart", cid, "is", total)
    # Checkout with discount
    discount  =  discount_transformer.read([did])
    if not discount.empty:
        total -= total * (discount[C.dpct].values[0] / 100)
    else:
        print(f"Discount {did} not found, aborting checkout. total value = ", total)
        return 
    purchase_entry = [{
        C.prcid: prcid,
        C.cid: cid,
        C.tamt: total,
        C.did: did
    }]
    purchase_transformer.create(pd.DataFrame.from_records(purchase_entry))
    discount_transformer.update({C.did: did, C.dst: 1})
    return

def checkout_without_discount(uid, cid, did, discode, dpct):
    cart      =  cart_transformer.read([cid])
    ps  =  product_transformer.read(cart[C.pid].to_list())

    # Compute total price for the cart
    total = ps[C.mrp].sum() * cart[C.qnt].sum()
    prcid = uid + "_" + cid  # Purchase ID based on user and cart ID
    print("Total price for cart", cid, "is", total)
    # Checkout with discount
    purchase_entry = [{
        C.prcid: prcid,
        C.cid: cid,
        C.tamt: total,
        C.did: did
    }]

    purchase_transformer.create(pd.DataFrame.from_records(purchase_entry))

    discount_entry = [{
        C.did: did,
        C.usrid: uid,
        C.dcode: discode,
        C.dpct: dpct,
        C.dst: 0
    }]
    discount_transformer.create(pd.DataFrame.from_records(discount_entry))
    return

# **Admin APIs**

In [ ]:
def getPurchases():
    purchases = purchase_transformer.read()
    print(purchases) 

def getDiscounts():
    discounts = discount_transformer.read()
    print( discounts )

getPurchases()
getDiscounts()

# **Use cases 1 - Buy products**

In [ ]:
# Create a random data
cid = "CART" + str(random.randint(100, 999))  # Generate a random cart ID
user_id = str(random.randint(100, 999))
discode = [ "NEW", "SAVE", "FESTIVE"][random.randint(0, 2)] + str(random.randint(1000, 9999))
did = ["FD","NC","DR"][random.randint(0,2)] + str(random.randint(100, 999))
dpct = random.randint(5, 10)

# Business logic using Helper functions
sps = getRandomPids()
print("Selected \n", sps)

addToCart(sps, user_id, cid)
print("Cart created and products added. \n")
cart = cart_transformer.read([cid])
print(cart)

checkout_without_discount(user_id, cid, did, discode, dpct)
print ( purchase_transformer.read([user_id+"_"+cid]) )

discounts = discount_transformer.read([did])
discounts

# **Use cases 2 - Buy products with discount**

In [ ]:
# Create a random data
cid = "CART" + str(random.randint(100, 999))  # Generate a random cart ID
user_id = str( random.randint(100, 999) )

# Business logic using Helper functions
sps = getRandomPids()
print("Selected \n", sps)

addToCart(sps, user_id, cid)
print("Cart created and products added. \n")
cart = cart_transformer.read([cid])
print(cart)

checkout_with_discount(user_id, cid, did)
print ( purchase_transformer.read([user_id+"_"+cid]) )

discounts = discount_transformer.read([did])
discounts